<a href="https://colab.research.google.com/github/king398/PestDetectFinal/blob/master/Train_YoloV5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Make Sure to Enable GPU in colab with the premium GPU class enabled

# Data Preparation
#Cell 1 - download data from google drive
#Cell 2 - download yolov5 and unzip the data
#Cell 3 - move data in the yolov5 dir 


In [ ]:
!pip3 install --upgrade gdown
import gdown
url = "https://drive.google.com/drive/folders/1fGD_2GEHZ327zybNYStKoI3WxAGPXWP4?usp=share_link"
gdown.download_folder(url, quiet=True, use_cookies=False)
gdown.download('https://drive.google.com/uc?export=download&id=12DPZdIHLlDXhGyasFFRntRtIVARY3j0o',quiet=False,output='Train.csv')
gdown.download('https://drive.google.com/u/0/uc?id=1NjBPcIjLJBeJ8YawWChsJxWQflLXkS22&export=download',quiet=False,output='train_modified.csv',fuzzy=True)


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
!unzip -qq /content/data/data.zip 
!unzip  -qq /content/data/test_images.zip


In [ ]:
!mv /content/data/dataset/ /content/data/fold_0.yaml /content/data/fold_1.yaml /content/data/fold_2.yaml /content/data/fold_3.yaml /content/data/fold_4.yaml /content/content/test_images yolov5
!rm -rf data /content/content

#YoloV5 Training
# Training yolov5l6 with image size 1536 for 25 epoch for each fold in the dataset (around 6 min per epoch take about 13 hours to complete with a A100)

In [ ]:
%cd yolov5
!python train.py --device 0 --epochs 25 --batch-size '-1' --data fold_0.yaml --img 1536 --weights 'yolov5l6.pt' --name yolov5l6-1536-image-size-25-epoch-fold-0 --seed 42 
!python train.py --device 0 --epochs 25 --batch-size '-1' --data fold_1.yaml --img 1536 --weights 'yolov5l6.pt' --name yolov5l6-1536-image-size-25-epoch-fold-1 --seed 42
!python train.py --device 0 --epochs 25 --batch-size '-1' --data fold_2.yaml --img 1536 --weights 'yolov5l6.pt' --name yolov5l6-1536-image-size-25-epoch-fold-2 --seed 42   
!python train.py --device 0 --epochs 25 --batch-size '-1' --data fold_3.yaml --img 1536 --weights 'yolov5l6.pt' --name yolov5l6-1536-image-size-25-epoch-fold-3 --seed 42   
!python train.py --device 0 --epochs 25 --batch-size '-1' --data fold_4.yaml --img 1536 --weights 'yolov5l6.pt' --name yolov5l6-1536-image-size-25-epoch-fold-4 --seed 42   


# Generate OOF preds For yolov5


In [ ]:
import os
import glob
import shutil

save_path = "/content/yolov5l6-1536-image-size-25-epoch-mskf"
for i in range(5):
    os.system(
        f"python detect.py --weights /content/yolov5/runs/train/yolov5l6-1536-image-size-25-epoch-fold-{i}/weights/best.pt "

        f"--img-size 1536 --half "
        f"--source dataset/fold_{i}/images "
        f"--name yolov7x-custom-different-augs-image-size-1024-{i}_val --save-txt "
        f"--conf 0.1 "
        f"--save-conf "
        f"   --nosave --augment ")
preds_txt = glob.glob(
    'runs/detect/yolov7x-custom-different-augs-image-size-1024-*_val/labels/*.txt')

os.makedirs(save_path,
            exist_ok=True)
for i in preds_txt:
    shutil.copy(i,
                save_path)
    os.remove(i)

# Generate Inference preds for fold 4 of the the yolov5 model (performs best on LB
)

In [ ]:
import glob
import os
import shutil

save_dir = '/content/yolov5l6-1536-image-size-25-epoch'
os.makedirs(save_dir, exist_ok=True)  

!python detect.py --half --nosave --weights  /content/yolov5/runs/train/yolov5l6-1536-image-size-25-epoch-fold-4/weights/best.pt --img-size 1536 --source test_images --name fold_4_test --save-txt --conf 0.01 --save-conf --augment


dirs = os.listdir('runs/detect/')
for i in dirs:
    shutil.move(f'runs/detect/{i}/', f'{save_dir}')

os.system('rm -r runs/detect/')


In [ ]:
%cd ..

# Make A binary classifier that detects if there are Bollworms present in the image 

In [ ]:
import glob
import os
import random
import cv2
import numpy as np
import torch
from sklearn.metrics import roc_auc_score
from torch import nn
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

os.system('pip install -qq wandb timm albumentations gpustat')

import timm
from albumentations import *
from albumentations.pytorch import ToTensorV2
import wandb
dataset_path = f''
os.makedirs('classfication/tf_efficientnet_b2_ns', exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wandb.init(project="pesticide")


def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)


set_seed(42)


class BollwormDataset(Dataset):
    def __init__(self, paths, transforms=None):
        super().__init__()
        self.paths = paths
        self.transforms = transforms

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index, ):
        path = self.paths[index]

        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transforms:
            img = self.transforms(image=img)['image']
        id = path.split('/')[-1].split('.')[0]
        fold = path.split('/')[-3]
        if os.path.exists(f'/content/yolov5/dataset/{fold}/labels/{id}.txt'):
            label = 1
        else:
            label = 0
        return img, torch.tensor(label)


def transform(DIM):
    return Compose([
        Resize(DIM, DIM),

        Normalize(),
        ToTensorV2()

    ])


class Model(nn.Module):
    def __init__(self, model_name, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, num_classes=1)
        self.model.set_grad_checkpointing(True)

    def forward(self, x):
        x = self.model(x)
        return x


def roc_auc_pytorch(y_true, y_pred):
    """Return the roc auc given a pytorch input and a pytorch target"""
    return roc_auc_score(y_true.detach().cpu().numpy(), y_pred.detach().cpu().numpy())


def accuracy(output, target, threshold=0.3):
    output = (output > threshold).int()
    return (output == target).float().mean()


def train(model, optimizer, loader, criterion, scheduler, device, fold):
    model.train()
    final_loss = 0
    scaler = GradScaler()
    stream = tqdm(loader, total=len(loader))
    preds = []
    targets = []
    for data, target in stream:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        with autocast():
            output = model(data)
            loss = criterion(output, target.unsqueeze(1).float())
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        final_loss += loss.item()

        scheduler.step()
        preds.append(output.sigmoid().detach().cpu())
        targets.append(target.detach().cpu())
        stream.set_description(f"Train Loss {loss.item() / len(stream):.4f}")
    wandb.log({f"Train Loss Fold {fold}": final_loss / len(loader)})


def val(model, loader, criterion, device, fold):
    model.eval()
    final_loss = 0
    preds = []
    targets = []
    stream = tqdm(loader, total=len(loader))
    with torch.no_grad():
        for data, target in stream:
            data, target = data.to(device), target.to(device)
            with autocast():
                output = model(data)
                loss = criterion(output, target.unsqueeze(1).float())
            final_loss += loss.item()
            preds.append(output.sigmoid().detach().cpu())
            targets.append(target.detach().cpu())
            stream.set_description(f"Val Loss {loss.item() / len(stream):.4f}")
        roc_auc = roc_auc_pytorch(torch.cat(targets), torch.cat(preds))
        accuracy_score = accuracy(torch.cat(preds), torch.cat(targets))
        print(f"Val Loss {final_loss / len(loader)} ROC AUC {roc_auc:.6f} Accuracy {accuracy_score:.6f}")
        wandb.log({f"Val Loss fold {fold}": final_loss / len(loader), f"ROC AUC {fold} ": roc_auc})
        return roc_auc


folds = [0, 1,2,3,4 ]
for i in folds:
    fold_num = i
    print(f'Fold {i}')
    path = []
    for j in range(5):
        if j != i:
            path.append(f'/content/yolov5/dataset/fold_{j}')
    val_path = f'/content/yolov5/dataset/fold_{i}'

    train_ids_full_path = []
    for i in path:
        p = glob.glob(f'{i}/images/*.jpg') + glob.glob(f'{i}/images/*.jpeg')
        train_ids_full_path.extend(p)
    train_ids = [i.split('/')[-1].split('.')[0] for i in train_ids_full_path]
    val_ids_full_path = glob.glob(f"{val_path}/images/*.jpg") + glob.glob(f"{val_path}/images/*.jpeg")
    val_ids = [i.split('/')[-1].split('.')[0] for i in val_ids_full_path]
    train_ds = BollwormDataset(train_ids_full_path, transforms=transform(1024))
    val_ds = BollwormDataset(val_ids_full_path, transforms=transform(1024))
    print(val_ids_full_path[0])
    train_dl = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=8, pin_memory=True)
    val_dl = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=8, pin_memory=True)
    model = Model('tf_efficientnet_b2_ns', pretrained=True)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.BCEWithLogitsLoss()
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3, steps_per_epoch=len(train_dl), epochs=10)

    for epoch in range(10):
        print(f"Epoch {epoch} Started")
        train(model, optimizer, train_dl, criterion, scheduler, device, i)
        val(model, val_dl, criterion, device, i)

    torch.save(model.state_dict(), f"/content/classfication/tf_efficientnet_b2_ns/model_{fold_num}.pth")


# Make oof predictions For Binary Classifier

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import glob
import os
import random
import cv2
import numpy as np
import timm
import torch
from albumentations import *
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm


def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)


set_seed(42)


class Model(nn.Module):
    def __init__(self, model_name, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, num_classes=1)

    def forward(self, x):
        x = self.model(x)
        return x


class BollwormDataset(Dataset):
    def __init__(self, path, ids, val, transforms=None):
        super().__init__()
        self.path = path
        self.ids = ids
        self.val = val
        self.transforms = transforms

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index, ):
        path = self.ids[index]
        id = path.split('/')[-1].split('.')[0]

        img = cv2.imread(path)
        if os.path.exists(f'{self.path}/labels/{id}.txt'):
            label = 1
        else:
            label = 0
        if self.transforms:
            img = self.transforms(image=img)['image']

        return img, torch.tensor(label)


def oof_fn(model, device, loader, ):
    model.eval()

    preds = []
    loader = tqdm(loader)
    with torch.no_grad():
        for img, label in loader:
            img = img.to(device)
            with torch.cuda.amp.autocast():
                pred = model(img)
            preds.append(pred.sigmoid().detach().cpu().numpy())
    preds = np.concatenate(preds)
    preds = preds.reshape(-1)
    return preds


final_val = []
final_pred = []
for i in range(5):
    print(f'Fold {i}')
    path = f"/content/yolov5/dataset/fold_{i}"
    val_ids = glob.glob(f"{path}/images/*")

    val_dataset = BollwormDataset(path, val_ids, val=True, transforms=Compose([
        Resize(1024, 1024),
        Normalize(),
        ToTensorV2()
    ]))
    val_ids = [i.split('/')[-1].split('.')[0] for i in val_ids]
    final_val.extend(val_ids)
    val_dl = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=8, pin_memory=True)
    model = Model('tf_efficientnet_b2_ns', pretrained=False)
    model.load_state_dict(torch.load(
        f'/content/classfication/tf_efficientnet_b2_ns/model_{i}.pth'))
    model = model.cuda()
    device = torch.device('cuda')
    val_preds = oof_fn(model, device, val_dl)
    final_pred.extend(val_preds)

oof_df = pd.DataFrame()
oof_df['id'] = final_val
oof_df['pred'] = final_pred
oof_df.to_csv('tf_effnet_b2_1024_image_size.csv',
              index=False)


# Inference For it


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import glob
import os
import random
import cv2
import numpy as np
import timm
import torch
from albumentations import *
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm


def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)


set_seed(42)


class Model(nn.Module):
    def __init__(self, model_name, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, num_classes=1)

    def forward(self, x):
        x = self.model(x)
        return x


class BollwormDataset(Dataset):
    def __init__(self, path, ids, transforms=None):
        super().__init__()
        self.path = path
        self.ids = ids
        self.transforms = transforms

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index, ):
        id = self.ids[index]
        img = cv2.imread(f'{self.path}/{id}.jpg')
        if img is None:
            img = cv2.imread(f'{self.path}/{id}.jpeg')

        if self.transforms:
            img = self.transforms(image=img)['image']

        return img


def inference(model, loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for images in tqdm(loader):
            images = images.to(device)
            with torch.cuda.amp.autocast():
                pred = model(images)
            preds.append(pred.sigmoid().detach().cpu().numpy())

    return np.concatenate(preds).reshape(-1)


preds = None
ids = glob.glob('/content/yolov5/test_images/*')
ids = [os.path.basename(id) for id in ids]
ids = [id.split('.')[0] for id in ids]
dataset = BollwormDataset('/content/yolov5/test_images', ids,
                          transforms=Compose([
                              Resize(1024, 1024),
                              Normalize(),
                              ToTensorV2()
                          ]))
loader = DataLoader(dataset, batch_size=16, shuffle=False, num_workers=8, pin_memory=True)
for i in range(5):
    model = Model('tf_efficientnet_b2_ns', pretrained=False)
    model.load_state_dict(
        torch.load(
            f'/content/classfication/tf_efficientnet_b2_ns/model_{i}.pth'))
    model.to('cuda')
    pred = inference(model, loader, 'cuda')
    if preds is None:
        preds = pred / 5
    else:
        preds += pred / 5
    del pred

df = pd.DataFrame()
df['id'] = ids
df['label'] = preds
df.to_csv('tf_effnet_b2_1024_image_size_inference.csv',
          index=False)


# Optuna Optimize on OOF to get best params

In [ ]:
!pip install ensemble-boxes pybboxes optuna

In [ ]:
import optuna
import pandas as pd
import os
import numpy as np
from statistics import mean
from ensemble_boxes import *

from pybboxes import BoundingBox
from joblib import Parallel, delayed
from tqdm.auto import tqdm
import yaml

train_df = pd.read_csv('Train.csv')
train_labels_df = pd.read_csv('train_modified.csv')
ids = []
labels = []
pred_labels_path = '/content/yolov5l6-1536-image-size-25-epoch-mskf'
id_label_dict = dict(zip(train_labels_df['image_id'].values, train_labels_df['number_of_worms'].values))

classifier_pred = pd.read_csv(
    'tf_effnet_b2_1024_image_size.csv')
classifier_pred_dict = dict(zip(classifier_pred['id'].values, classifier_pred['pred'].values))


def mae(y_true, y_pred):
    return np.abs(y_true - y_pred)


def return_error(id, pred_label_dict):
    id = id.split('.')[0]
    pbw_label = id_label_dict[f'{id}_pbw.jpg']
    abw_label = id_label_dict[f'{id}_abw.jpg']
    pbw_pred = pred_label_dict[f'{id}_pbw.jpg']
    abw_pred = pred_label_dict[f'{id}_abw.jpg']
    error = float(
        mae(np.array(pbw_label), np.array(pbw_pred)) + float(mae(np.array(abw_label), np.array(abw_pred))))
    return error


def make_labels(id, params):
    id = id.split('.')[0]
    pbw = 0
    abw = 0

    classifier_pred = classifier_pred_dict[id] * 1.0

    if os.path.exists(
            f'{pred_labels_path}/{id}.txt') and classifier_pred > params['classifier_thresh']:
        with open(
                f'{pred_labels_path}/{id}.txt') as f:
            preds_per_line = f.readlines()
            bboxes = []
            scores = []
            label = []

            for i in preds_per_line:
                i = i.split(' ')
                bbox = [float(i[1]), float(i[2]), float(i[3]), float(i[4])]
                bbox = BoundingBox.from_yolo(*bbox, image_size=(1536,1536))
                bbox = bbox.to_albumentations().values

                bboxes.append(list(bbox))
                scores.append(float(i[5]))

                label.append(int(i[0]))

            bboxes, scores, label = soft_nms([bboxes], [scores], [label], iou_thr=params['iou_thr'],
                                             sigma=params['sigma'], thresh=params['thresh'], method=params['method'], )

            for i in range(len(label)):
                if label[i] == 0:
                    pbw += 1
                else:
                    abw += 1

    return pbw, abw, f"{id}_pbw.jpg", f"{id}_abw.jpg"


class error_func:
    def __init__(self, pred_label_dict):
        super().__init__()
        self.pred_label_dict = pred_label_dict

    def return_error(self, id):
        id = id.split('.')[0]
        pbw_label = id_label_dict[f'{id}_pbw.jpg']
        abw_label = id_label_dict[f'{id}_abw.jpg']
        pbw_pred = self.pred_label_dict[f'{id}_pbw.jpg']
        abw_pred = self.pred_label_dict[f'{id}_abw.jpg']
        error = float(
            mae(np.array(pbw_label), np.array(pbw_pred)) + float(mae(np.array(abw_label), np.array(abw_pred))))
        return error


def objective(trial):
    params = {
        'iou_thr': trial.suggest_float('iou_thr', 0.1, 0.7),
        'sigma': trial.suggest_float('sigma', 0.3, 1.0),
        'thresh': trial.suggest_float('thresh', 0.2, 0.6),
        'method': trial.suggest_categorical('method', ['nms', 'linear', 'gaussian']),
        'classifier_thresh': trial.suggest_float('classifier_thresh', 0.1, 0.7),

    }

    pred = Parallel(n_jobs=12)(delayed(make_labels)(id, params) for id in tqdm(train_df['image_id_worm'].values))
    ids = []
    labels = []
    for i in pred:
        ids.append(i[2])
        ids.append(i[3])
        labels.append(i[0])
        labels.append(i[1])
    oof = pd.DataFrame({'image_id_worm': ids, 'label': labels}, index=None)

    pred_label_dict = dict(zip(oof['image_id_worm'].values, oof['label'].values))
    error_fn = error_func(pred_label_dict)

    error = list(map(error_fn.return_error, train_df['image_id_worm'].values))

    return mean(error)


study = optuna.create_study(direction='minimize',)
study.optimize(objective, n_trials=2000, )
best_param_save = study.best_params
best_param_save.update({'best_score': study.best_value})
best_param_save.update({'best_trial': study.best_trial.number})
best_param_save.update({'path': pred_labels_path})
### with best_param_save.yaml to 
with open(
        f'{pred_labels_path.split("/")[-1]}.yaml',
        'w') as f:
    yaml.dump(best_param_save, f)


In [ ]:
# Import libraries
## Reference https://zindi.africa/learn/how-to-download-data-files-from-zindi-to-colab
import requests
from tqdm.auto import tqdm

# Data url and token
data_url_test = "https://api.zindi.africa/v1/competitions/wadhwani-ai-bollworm-counting-challenge/files/Test.csv"


token = {'auth_token': ''}  # Use your own token


def zindi_data_downloader(url, token, file_name):
    # Get the competition data
    competition_data = requests.post(url=url, data=token, stream=True)

    # Progress bar monitor download
    pbar = tqdm(desc=file_name, total=int(competition_data.headers.get('content-length', 0)), unit='B', unit_scale=True,
                unit_divisor=512)
    # Create and Write the data to colab drive in chunks
    handle = open(file_name, "wb")
    for chunk in competition_data.iter_content(chunk_size=512):  # Download the data in chunks
        if chunk:  # filter out keep-alive new chunks
            handle.write(chunk)
        pbar.update(len(chunk))
    handle.close()
    pbar.close()


zindi_data_downloader(url=data_url_test, token=token, file_name='Test.csv')


# Run inference pred with the best params obtained by optuna

In [ ]:
__author__ = 'Mithil Salunkhe: https://www.kaggle.com/mithilsalunkhe'

from statistics import mode
from ensemble_boxes import nms
import numpy as np
import pandas as pd
import os
from ensemble_boxes import *
from tqdm import tqdm
from pybboxes import BoundingBox
import yaml

test_df = pd.read_csv('Test.csv')
pred_path = f'/content/yolov5l6-1536-image-size-25-epoch'
classifier_df = pd.read_csv(
    'tf_effnet_b2_1024_image_size_inference.csv')
classifer_dict = dict(zip(classifier_df['id'].values, classifier_df['label'].values))
ids = []
labels_final = []
with open(
        'yolov5l6-1536-image-size-25-epoch-mskf.yaml') as f:
    params = yaml.safe_load(f)


def make_labels(id, params=params):
    pbw = 0
    abw = 0
    id = id.split('.')[0]

    ids.extend([f"{id}_pbw", f"{id}_abw"])

    classifier_pred = classifer_dict[id]
    bboxes = []
    labels = []
    scores = []
    pbw_list = []
    abw_list = []
    for i in range(1):
        pbw = 0
        abw = 0
        i = 4

        labels_temp = []
        bbox_temp = []
        score_temp = []
        path = f'{pred_path}/fold_{i}_test/labels/{id}.txt'

        if os.path.exists(path) and classifier_pred > params['classifier_thresh']:
            with open(path) as f:
                preds_per_line = f.readlines()

                for i in preds_per_line:
                    i = i.split(' ')
                    bbox = [float(i[1]), float(i[2]), float(i[3]), float(i[4])]
                    try:
                        bbox = BoundingBox.from_yolo(*bbox, image_size=(1536, 1536))
                        bbox = bbox.to_albumentations().values

                        bbox_temp.append(bbox)
                        score_temp.append(float(i[5]))
                        labels_temp.append(int(i[0]))
                    except:
                        pass
                bbox_temp, score_temp, labels_temp = soft_nms([bbox_temp], [score_temp], [labels_temp],
                                                              iou_thr=params['iou_thr'],
                                                              sigma=params['sigma'], thresh=params['thresh'],
                                                              method=params['method'])
                bboxes.append(bbox_temp)
                scores.append(score_temp)
                labels.append(labels_temp)
        for i in labels_temp:
            if i == 0:
                pbw += 1
            else:
                abw += 1
        pbw_list.append(pbw)
        abw_list.append(abw)
    pbw = int(np.mean(pbw_list))
    abw = int(np.mean(abw_list))
    labels_final.extend([pbw, abw])


list(map(make_labels, tqdm(test_df['image_id_worm'].values)))
submission = pd.DataFrame({'image_id_worm': ids, 'label': labels_final}, index=None)
submission.to_csv(
    'yolov5l6-1536-image-size-25-epoch.csv',
    index=False)
